# TP-2 Organización de Datos

# Oh La La... Data!

* **Hamma AALI CHTOUKI** --- *Padrón 106607 - Alumno regular de intercambio.*  
* **Aymeryc COUSAERT** ------  *Padrón 105464 - Alumno regular de intercambio.*
* **Mariana VINYOLAS**  ---------  *Alumna oyente.*

<!-- ![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
<a id='section2'></a>

<div style="border-bottom:3px solid #000;"> 
<div align="left"><h2>Step 2: Import de librerias </h2></div>
<div align="right">(return to <a href='#top1'><b>Top</b></a>)</div>
</div> -->



In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns


import re
import nltk.corpus
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import catboost as catb
import lightgbm as lgbm

import scikitplot as skplt
from scikitplot.metrics import plot_roc
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

C:\anaconda\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


<!-- ![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
<a id='section3'></a>

<div style="border-bottom:3px solid #000;"> 
<div align="left"><h2>Step 3: Feature engineering</h2></div>
<div align="right">(return to <a href='#top1'><b>Top</b></a>)</div>
</div> -->

In [5]:

df = train
df['text'] = df['text'].str.lower()   # pasamos a lowercase
df = df.drop(['id', 'location'], axis=1)

In [6]:
df.sample(8)

,keyword,text,target
3852,flames,*new* snap on tools black baseball hat/cap sil...,0
588,bioterror,fedex no longer to transport bioterror germs i...,1
5301,outbreak,new york city outbreak: what is legionnaire's ...,1
5016,mudslide,'it looks like a mudslide' 'it's like chewing ...,0
3914,flood,spot flood combo 53inch 300w curved cree led w...,0
6426,suicide%20bomber,#?? #?? #??? #??? suicide bomber kills 15 in s...,1
5790,rioting,`bbcnews the ass. of british insurers says rio...,1
2988,drowning,@homukami only urs and srs matter rs you'll be...,0


#### Generamos nuevas features sobre el texto

In [7]:
# generamos una columna que indica la cantidad de links a  enlaces externos
df['link'] = df['text'].apply(lambda x: x.count('http'))

# generamos una columna que indica la cantidad de referencias a otras cuentas de twitter
df['contact'] = df['text'].apply(lambda x: x.count('@'))

# generamos una columna que indica la cantidad de hashtags
df['hashtag'] = df['text'].apply(lambda x: x.count('#'))

# generamos una columna que indica la cantidad de digitos
df['numerics'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

# calculamos la longitud del tweet andes de limpiar
df['length'] = df['text'].str.len()

In [8]:
# calculamos la cantidad de palabras antes de limpiar
def count_words(text):
    '''
    Funcion que toma un texto y devuelve la cantidad de palabras
    '''
    word_counts = len(text.split(' '))
    return word_counts

df['words'] = df['text'].apply(count_words)

In [9]:
# limpiamos el texto eliminando urls, cuentas, hashtags y numeros

def clean_str(string):
    string = re.sub(r'https?\://\S+', '', string)
    string = re.sub(r'http?\://\S+', '', string)
    string = re.sub(r'@\w*\s', '', string)
    string = re.sub(r'#\w*\s', '', string)
    string = re.sub(r'\d', '', string)
    return string

df['text_clean'] = df['text'].apply(lambda x: clean_str(str(x)))

In [10]:
df.sample(5)

,keyword,text,target,link,contact,hashtag,numerics,length,words,text_clean
1829,crashed,pakistan says army helicopter has crashed in c...,1,1,0,0,1,114,15,pakistan says army helicopter has crashed in c...
1458,casualty,#nowplaying: dubstep hardstyle trap messy mix ...,1,2,1,1,0,138,15,#nowplaying: dubstep hardstyle trap messy mix ...
2611,destroyed,black eye 9: a space battle occurred at star o...,0,0,0,0,3,106,19,black eye : a space battle occurred at star o ...
2213,deluge,it's a deluge in trois-rivieres. about one hou...,0,1,0,1,0,102,13,it's a deluge in trois-rivieres. about one hou...
6974,tsunami,@greenlacey godslove &amp; #thanku my sister f...,0,1,1,1,0,138,19,godslove &amp; my sister for rt of new video ...


<!-- ![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
<a id='section4'></a>

<div style="border-bottom:3px solid #000;"> 
<div align="left"><h2>Step 4: NLP</h2></div>
<div align="right">(return to <a href='#top1'><b>Top</b></a>)</div>
</div> -->

In [11]:
# eliminamos stopwords

stop = stopwords.words('english')

df['text_clean'] = df['text_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [12]:
# eliminamos signos de puntuacion y caracteres especiales
df['text_clean'] = df['text_clean'].str.replace('[^\w\s]','')

In [13]:
df['word_tokenize'] = df['text_clean'].apply(lambda x: word_tokenize(x))

In [14]:
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

In [15]:
# unificamos palabras que poseen la misma raiz aplicando la funcion word_lematizer
df['word_lemmatizer'] = df['word_tokenize'].apply(lambda x: word_lemmatizer(x))

In [16]:
# unificamos la lista de tokens para poder analizar el texto limpio
df['text_clean'] = df['word_lemmatizer'].str.join(' ')

In [17]:
# calculamos nuevamente la longuitud, pero ahora del texto limpio                       
df['length-clean'] = df['text_clean'].str.len()

In [18]:
df['words_clean'] = df['text_clean'].apply(count_words)

In [19]:
pd.set_option('display.max_colwidth', 150)  

In [20]:
df[['text', 'text_clean']].sample(10)

,text,text_clean
2522,going to redo my nails and watch behind the scenes of desolation of smaug ayyy,going redo nail watch behind scene desolation smaug ayyy
3792,@justinejayyy ohgod xd i didn't mean it so =p but you have that fire truck in the back of you to make up for it so you good xd,ohgod xd mean p fire truck back make good xd
6860,@thetimepast @saalon i have childhood trauma more resolved than theirs. actual trauma. fricken babies.,childhood trauma resolved theirs actual trauma fricken baby
4079,thank you so so much to everyone for posting the rain and hail outside ... i had no idea guys ????????,thank much everyone posting rain hail outside idea guy
4462,#hot c-130 specially modified to land in a stadium and rescue hostages in iran in 1980 http://t.co/w0exzad5gc #prebreak #best,c specially modified land stadium rescue hostage iran best
2866,tips so that finding the customers ego drought: dqsvyusy,tip finding customer ego drought dqsvyusy
3487,@aminespn mencius tears are worse correct? takes the explosion n more pain day to day right?,mencius tear worse correct take explosion n pain day day right
6523,you can't fight fate and you can't survive alone... i can't help but notice that almost seems like a definition of who i am...,cant fight fate cant survive alone cant help notice almost seems like definition am
2575,@sapphirescallop destroy oppa image? oops! there's nothing left right? haaaaaa,destroy oppa image oops there nothing left right haaaaaa
1576,i hope they fall off a cliff.,hope fall cliff


In [21]:
df['keyword'] = df.keyword.str.replace('%20', '_')
df['keyword'] = df.keyword.str.lower()

In [22]:
df.sample(3)

,keyword,text,target,link,contact,hashtag,numerics,length,words,text_clean,word_tokenize,word_lemmatizer,length-clean,words_clean
4872,mass_murderer,not only are you a mass murderer but at a movie theatre where niggas dropped bread to see a movie? cmon man.,0,0,0,0,0,108,22,mass murderer movie theatre nigga dropped bread see movie cmon man,"[mass, murderer, movie, theatre, niggas, dropped, bread, see, movie, cmon, man]","[mass, murderer, movie, theatre, nigga, dropped, bread, see, movie, cmon, man]",66,11
4207,hazard,battlefield 4 funny moments - dukes of hazard undercover soldier mav t... https://t.co/ju8nfpnedl via @youtube,0,1,1,0,1,110,15,battlefield funny moment duke hazard undercover soldier mav t via youtube,"[battlefield, funny, moments, dukes, hazard, undercover, soldier, mav, t, via, youtube]","[battlefield, funny, moment, duke, hazard, undercover, soldier, mav, t, via, youtube]",73,11
6499,survive,hank williams jr. - 'country boys can survive' (official music video) https://t.co/you7wn9xvs via @youtube,0,1,1,0,0,106,14,hank williams jr country boy survive official music video via youtube,"[hank, williams, jr, country, boys, survive, official, music, video, via, youtube]","[hank, williams, jr, country, boy, survive, official, music, video, via, youtube]",69,11


In [23]:
df['target'].value_counts()  # chequeamos si los labels estan balanceados

0    4342
1    3271
Name: target, dtype: int64

In [24]:
# eliminamos col que contienen texto y conservamos unicamente las numericas
df_num = df.drop([ 'keyword', 'text', 'text_clean', 'word_tokenize', 'word_lemmatizer'], axis=1)

## Train Test Split

In [25]:
# separamos el target del resto de los features

y = df_num.target    
X = df_num.drop('target', axis=1)

In [26]:
X.shape, y.shape

((7613, 8), (7613,))

In [27]:
# hacemos division entre train y test para cross validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)      

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5329, 8), (2284, 8), (5329,), (2284,))

## Random Forest

In [28]:
rf = RandomForestClassifier(max_depth= 4, random_state=27)
y_pred = rf.fit(X_train, y_train).predict(X_test)

In [29]:
rf.score(X_train, y_train), rf.score(X_test,y_test)

(0.6637267780071308, 0.6598073555166375)

Con un score tan bajo probamos hacer un GridSearch para correr el modelo con mejores parametros

## GridSearch

In [30]:
# definimos cross validation
cv = StratifiedKFold(n_splits=5, random_state=27, shuffle=True)

In [31]:
param_grid = {'n_estimators': [ 200, 250, 300], 
              'max_features': [1, 2, 3], 
              'max_depth': [ 10, 20, 30], 
              'min_samples_leaf':[5, 10, 20]}


In [32]:
rf_grid_search = GridSearchCV(RandomForestClassifier(random_state=0), param_grid=param_grid,
                              cv=cv, verbose=1, n_jobs=-1, scoring='accuracy')

In [33]:
%%time
rf_grid_search.fit(X_train, y_train)  


Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   29.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:  4.8min finished


Wall time: 4min 53s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=27, shuffle=True),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                            

In [34]:
rf_grid_search.best_params_

{'max_depth': 30,
 'max_features': 2,
 'min_samples_leaf': 5,
 'n_estimators': 250}

In [35]:
pred_grid = rf_grid_search.predict(X_test)

In [36]:
rf_grid_search.score(X_train, y_train), rf_grid_search.score(X_test,y_test)

(0.7967723775567649, 0.6843257443082311)

In [37]:
print(accuracy_score(y_test, pred_grid))   # sacando TF-IDF

0.6843257443082311


In [38]:
0.6554290718038529  # con TF IDF

0.6554290718038529